# UDF Installer
This notebook demonstrates how to use `pyTigerGraph` to install User Defined Functions (UDFs). UDFs are required for some graph algorithms and the data loaders (introduced later) to work. See https://docs.tigergraph.com/gsql-ref/current/querying/func/query-user-defined-functions for details on UDFs. 

**Note**: Installing UDF will overwrite any existing UDF in the database. Please back up your existing UDFs before installation.

### Connection to Database

The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. It is able to perform quite a few database tasks. Please see its [documentation](https://docs.tigergraph.com/pytigergraph/current/intro/) for details.

**Note**: Secret instead of username/password is required for TG cloud DBs created after 7/5/2022. Otherwise, you can leave it blank.

In [ ]:
from pyTigerGraph import TigerGraphConnection

In [ ]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

<span style="color:red">Uncomment cell below and run to get and set token if token authentication is enabled</span>. 
* This is required for all databases on tgcloud.
* `<secret>` is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.
* If you don't know your secret, you can use `secret=conn.createSecret()` to create one.

In [ ]:
#conn.getToken(<secret>)

## Install UDF
Install the desired UDF by providing the address where it is located. The address can be the path to a local file or the URL of a file online. Below we demonstrate the usage of the UDF installer by installing the UDFs for the builtin graph algorithms and the data loaders respectively. But don't run the two subsections below at the same time, as they will overwrite each other! Rather, install the UDF for graph algorithms when you need to use the featurizer, or the UDF for data loaders when you need the data loaders.

### UDF for data loaders

In [ ]:
ExprFunctions="https://tg-mlworkbench.s3.us-west-1.amazonaws.com/udf/1.0/ExprFunctions.hpp"  # For enterprise users, please use the link you received.
ExprUtil=""  # For enterprise users, please use the link you received.
conn.installUDF(ExprFunctions, ExprUtil)

In [ ]:
neighbor_loader = conn.gds.neighborLoader(
    num_batches=10,
    num_neighbors = 10,
    num_hops =2,
    v_in_feats = ["x"],
    v_out_labels = ["y"],
    v_extra_feats = ["train_mask", "val_mask", "test_mask"],
    e_in_feats=["time"],
    e_out_labels=[],
    e_extra_feats=["is_train", "is_val"],
    output_format = "PyG",
    shuffle=True,
    filter_by=None
)

In [ ]:
%%time
for i, batch in enumerate(neighbor_loader):
    print("----Batch {}----".format(i))
    print(batch)

### UDF for builtin graph algorithms (Optional)

**Note**: Those UDFs come pre-installed with TigerGraph database 3.6 and above. Do NOT run the cell below to reinstall the UDFs or you will get an error. However, if you are using database 3.5 and below, those UDFs are required.

In [ ]:
ExprFunctions="https://raw.githubusercontent.com/tigergraph/gsql-graph-algorithms/ML_Workbench_3.5/UDF/tg_ExprFunctions.hpp"
ExprUtil="https://raw.githubusercontent.com/tigergraph/gsql-graph-algorithms/ML_Workbench_3.5/UDF/tg_ExprUtil.hpp"
conn.installUDF(ExprFunctions, ExprUtil)

With the UDF installd, we can use any builtin graph algorithm. Here we use fastRP as an example. The fastRP algorithm is available in GDS library called tg_fastRP under the class of embedding algorithms  https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/GraphML/Embeddings/FastRP/tg_fastRP.gsql. Now fastRP can be used to create features.

In [ ]:
f = conn.gds.featurizer()
f.installAlgorithm(
    "tg_fastRP", 
    global_change = False # If a global schema change is needed to add an attribute, set it to True.
)

In [ ]:
params = {'v_type': 'Paper', 'e_type': ['Cite','reverse_Cite'], 'weights': '1,1,2', 'beta': -0.85, 'k': 3, 'reduced_dim': 128, 
          'sampling_constant': 1, 'random_seed': 42, 'print_accum': False,'result_attr':"",'file_path' :""}
f.runAlgorithm('tg_fastRP', params, feat_name ="fastrp_embedding")